**⭐ 1. What This Pattern Solves**

Surrogate keys are system-generated unique identifiers used in dimension and fact tables.
Use-cases include:

Assigning a unique customer ID for analytics

Fact tables referencing dimension tables via surrogate keys

Handling slowly changing dimensions without using natural keys

They ensure uniqueness, stability, and decouple keys from source system changes.

**⭐ 2. SQL Equivalent**

In [0]:
%sql
SELECT ROW_NUMBER() OVER (ORDER BY customer_name) AS surrogate_key,
       customer_name, customer_email
FROM staging_customers

**⭐ 3. Core Idea**

Use monotonically_increasing_id() or row_number() over a deterministic order

Assign a unique ID to each row

Can be persisted as primary key in dimension/fact tables

**⭐ 4. Template Code (MEMORIZE THIS)**

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.sql.functions import monotonically_increasing_id

# Option 1: Row number over deterministic order
window_spec = Window.orderBy("some_column")
df_with_sk = df.withColumn("surrogate_key", row_number().over(window_spec))

# Option 2: Monotonically increasing ID (good for large datasets)
df_with_sk = df.withColumn("surrogate_key", monotonically_increasing_id())

**⭐ 5. Detailed Example**

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, monotonically_increasing_id

spark = SparkSession.builder.getOrCreate()

data = [("Alice", "alice@example.com"), ("Bob", "bob@example.com"), ("Charlie", "charlie@example.com")]
df = spark.createDataFrame(data, ["name", "email"])

# Using row_number
window_spec = Window.orderBy("name")
df_sk = df.withColumn("surrogate_key", row_number().over(window_spec))
df_sk.show()

# Using monotonically_increasing_id
df_sk2 = df.withColumn("surrogate_key", monotonically_increasing_id())
df_sk2.show()

**Step-by-step:**

Choose deterministic column(s) for row_number OR use monotonically_increasing_id

Assign unique surrogate key

Use key in downstream fact tables or joins

**⭐ 6. Mini Practice Problems**

Generate surrogate keys for customer dimension table.

Assign unique IDs for sessionized events.

Create surrogate keys for product catalog in incremental ETL.

**⭐ 7. Full Data Engineering Problem**

Scenario: Building a data warehouse for e-commerce. Source systems have inconsistent customer IDs.

Solution Approach:

Load staging customers

Assign surrogate key using row_number or monotonically increasing ID

Store in dimension table

Fact table references dimension table using surrogate key

Handle incremental loads by generating new surrogate keys only for new customers

Performance tip: monotonically_increasing_id() scales better for very large datasets; row_number() ensures deterministic ordering for reproducibility.

**⭐ 8. Time & Space Complexity**

Time: O(N) for monotonically_increasing_id(); O(N log N) for row_number() (due to sort)

Space: O(N) for storing ID column

Deterministic ordering is slower but ensures repeatable results.

**⭐ 9. Common Pitfalls**

Using monotonically_increasing_id() with join operations without care → IDs may not be consecutive

Forgetting deterministic order when using row_number() → IDs change across runs

Reassigning keys during incremental updates → breaks fact table references

Large datasets + row_number() over global sort → memory and shuffle issues